In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from functools import partial

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17, default_train_params_ala_nips_17

from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only,\
encoder_with_convs_and_symmetry_new, decoder_with_convs_only

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tflearn.layers.conv import avg_pool_1d
from pc_completions.evaluation import basic_comletion_measures

from tflearn.layers.conv import conv_1d, highway_conv_1d, avg_pool_1d, max_pool_1d
from tf_lab.fundamentals.layers import conv_1d_tranpose

In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
n_pc_points = 2048
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))

In [5]:
do_training = True
load_pretrained_model = False
load_epoch = None
random_seed = 42
loss = 'chamfer'
training_epochs = 2000
batch_size = 50
learning_rate = 0.0005

In [6]:
train_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt')
val_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt')
test_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt')

splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)
tr_files = splitter.load_splits(train_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(tr_files, n_threads=20, loader=snc_loader, verbose=True)
train_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
val_files = splitter.load_splits(val_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(val_files, n_threads=20, loader=snc_loader, verbose=True)
val_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
test_files = splitter.load_splits(test_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(test_files, n_threads=20, loader=snc_loader, verbose=True)
test_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.


In [ ]:
def enc_dec_templates(exp_id):
    if exp_id == 0:
        encoder = encoder_with_convs_and_symmetry_new
        decoder = decoder_with_convs_only

        encoder_args = {'n_filters': [64, 128, 128, 256, 32],
                        'filter_sizes': [9, 3, 2, 2, 2],
                        'strides': [3, 2, 1, 1, 1],
                        'b_norm': b_norm,
                        'pool': avg_pool_1d,
                        'pool_sizes': [6, 4, 4, 2, 2],
                        'verbose': True,
                        'symmetry': None
                    }

        decoder_args = {'n_filters': [128, 128, 64, 64, 3],
                        'filter_sizes': [2, 3, 3, 3, 6],
                        'strides': [2, 4, 4, 8, 8],
                        'b_norm': b_norm,
                        'non_linearity': None,
                        'conv_op': deconv_op,
                        'verbose': True
                    }

    

In [155]:

# decoder = decoder_with_fc_only
b_norm = True



# deconv_op = conv_1d
deconv_op = partial(conv_1d_tranpose, batch_size=batch_size)





# decoder_args = {'layer_sizes': [32, 128, 128, 2048*3],
#                 'b_norm': b_norm,
#                 'b_norm_finish': True,
#                 'non_linearity': None,
#                 'verbose': True
#                 }


layer = tf.placeholder(tf.float32, [None, n_pc_points, 3])
layer = encoder(layer, **encoder_args)
layer = decoder(layer, **decoder_args)

Building Encoder
encoder_conv_layer_0 conv params =  1792 bnorm params =  128
Tensor("AvgPool1D_15/Squeeze:0", shape=(?, 114, 64), dtype=float32)
output size: 7296 

encoder_conv_layer_1 conv params =  24704 bnorm params =  256
Tensor("AvgPool1D_16/Squeeze:0", shape=(?, 15, 128), dtype=float32)
output size: 1920 

encoder_conv_layer_2 conv params =  32896 bnorm params =  256
Tensor("AvgPool1D_17/Squeeze:0", shape=(?, 4, 128), dtype=float32)
output size: 512 

encoder_conv_layer_3 conv params =  65792 bnorm params =  512
Tensor("AvgPool1D_18/Squeeze:0", shape=(?, 2, 256), dtype=float32)
output size: 512 

encoder_conv_layer_4 conv params =  16416 bnorm params =  64
Tensor("AvgPool1D_19/Squeeze:0", shape=(?, 1, 32), dtype=float32)
output size: 32 

Building Decoder
decoder_conv_layer_0 conv params =  8320 bnorm params =  256
Tensor("decoder_conv_layer_0_bnorm/batchnorm/add_1:0", shape=(50, 2, 128), dtype=float32)
output size: 256 

decoder_conv_layer_1 conv params =  49280 bnorm params =

In [150]:
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings/temp')

conf = Conf(
            n_input = [n_pc_points, 3],
            loss = loss,
            training_epochs = training_epochs,
            batch_size = batch_size,
            denoising = False,
            learning_rate = learning_rate,
            train_dir = train_dir,             
            loss_display_step = 1,
            saver_step = None,
            z_rotate = False,
            encoder = encoder, 
            decoder = decoder,
            encoder_args = encoder_args,
            decoder_args = decoder_args
           )

conf.experiment_name = 'Test'
conf.held_out_step = 5
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

print ae.trainable_parameters()

Building Encoder
encoder_conv_layer_0 conv params =  1792 bnorm params =  128
Tensor("Test_1/Relu:0", shape=(?, 683, 64), dtype=float32)
output size: 43712 

encoder_conv_layer_1 conv params =  24704 bnorm params =  256
Tensor("Test_1/Relu_1:0", shape=(?, 342, 128), dtype=float32)
output size: 43776 

encoder_conv_layer_2 conv params =  32896 bnorm params =  256
Tensor("Test_1/Relu_2:0", shape=(?, 342, 128), dtype=float32)
output size: 43776 

encoder_conv_layer_3 conv params =  65792 bnorm params =  512
Tensor("Test_1/Relu_3:0", shape=(?, 342, 256), dtype=float32)
output size: 87552 

encoder_conv_layer_4 conv params =  16416 bnorm params =  64
Tensor("Test_1/Relu_4:0", shape=(?, 342, 32), dtype=float32)
output size: 10944 

Building Decoder
decoder_fc_0 FC params =  350240 bnorm params =  64
Tensor("Test_1/decoder_fc_0_bnorm/batchnorm/add_1:0", shape=(?, 32), dtype=float32)
output size: 32 

decoder_fc_1 FC params =  4224 bnorm params =  256
Tensor("Test_1/decoder_fc_1_bnorm/batchnor

In [127]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)    
    train_stats = ae.train(train_data, conf, log_file=fout, held_out_data=val_data)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.0578', 'loss=', '2.232838356')
('Epoch:', '0002', 'training time (minutes)=', '0.0508', 'loss=', '1.542780295')
('Epoch:', '0003', 'training time (minutes)=', '0.0516', 'loss=', '1.234779115')
('Epoch:', '0004', 'training time (minutes)=', '0.0585', 'loss=', '1.046120163')
('Epoch:', '0005', 'training time (minutes)=', '0.0507', 'loss=', '0.884027935')
('Held Out Data :', 'forward time (minutes)=', '0.0019', 'loss=', '0.820954820')
('Epoch:', '0006', 'training time (minutes)=', '0.0506', 'loss=', '0.743712132')
('Epoch:', '0007', 'training time (minutes)=', '0.0475', 'loss=', '0.620006267')
('Epoch:', '0008', 'training time (minutes)=', '0.0567', 'loss=', '0.512350684')
('Epoch:', '0009', 'training time (minutes)=', '0.0496', 'loss=', '0.419372054')
('Epoch:', '0010', 'training time (minutes)=', '0.0501', 'loss=', '0.341981055')
('Held Out Data :', 'forward time (minutes)=', '0.0008', 'loss=', '0.307026476')
('Epoch:', '0011', 'training

KeyboardInterrupt: 

In [96]:
for i in range(10):
    print ae._single_epoch_train(train_data, conf)
    print reconstruct_pclouds(ae, train_data.point_clouds, 510, compute_loss=True)[1]

(0.00070632771366154368, 3.2764317989349365)
0.000619403164179
(0.00070817585447398215, 2.84295392036438)
0.00061691453293
(0.00071282371392716534, 2.72479510307312)
0.000601568156261
(0.0007123221688822884, 2.686289072036743)
0.000614956626907
(0.0007036546918401576, 2.905812978744507)
0.000621547584764
(0.00070395093137884271, 2.658350944519043)
0.00060599925454
(0.00070694156254277277, 2.7218101024627686)
0.000599984718904
(0.00070774137578985617, 3.156014919281006)
0.000612763098718
(0.0007164171765274976, 2.7741189002990723)
0.00060548167607
(0.00070373827146123281, 2.814279079437256)
0.000609081925007


In [101]:
print reconstruct_pclouds(ae, train_data.point_clouds, 510, compute_loss=True)[1]

0.000609081949913


In [98]:
int(train_data.num_examples / batch_size)

115

In [100]:
train_data.num_examples / float(batch_size)

115.22

In [93]:
# from tf_lab.point_clouds.convenience import reconstruct_pclouds

# _, l = reconstruct_pclouds(ae, train_data.point_clouds, 1, compute_loss=True)
# print l

_, l = reconstruct_pclouds(ae, train_data.point_clouds, 33, compute_loss=True)
print l

0.000612877637805


In [ ]:
# decoder_args = {'n_filters': [6, 6, 12, 12, 3],
#                 'filter_sizes': [9, 3, 2, 2, 1],
#                 'strides': [8, 4, 4, 2, 2],
#                 'b_norm': b_norm,
#                 'non_linearity': tf.nn.relu,
#                 'b_norm_finish': True,
#                 'conv_op': conv_1d,
#                 'verbose': True,
#                 'padding': 'valid'
#                 }

# b_norm = True
# encoder_args = {'n_filters': [64, 128, 128, 256],
#                 'filter_sizes': [9, 3, 2, 2],
#                 'strides': [1, 1, 1, 1],
#                 'b_norm': b_norm,
#                 'pool': avg_pool_1d,
#                 'pool_sizes': [2, None, 2, None],
#                 'symmetry': None
#                 }

# decoder_args = {'n_filters': [6, 6, 12, 12],
#                 'filter_sizes': [2, 2, 4, 8],
#                 'strides': [1, 1, 1, 1],
#                 'b_norm': b_norm,
#                 'non_linearity': tf.nn.relu,
#                 'b_norm_finish': True,
#                 'conv_op': conv_1d,
#                 'verbose': True
#                 }


# encoder_args = {'n_filters': [64, 64, 128, 128, 256, 256, 32],
#                 'filter_sizes': [9, 8, 8, 5, 3, 3, 2],
#                 'strides': [1, 1, 1, 1, 1, 1, 1],
#                 'b_norm': b_norm,
#                 'pool': avg_pool_1d,
#                 'pool_sizes': [4, 4, 4, 4, 2, 2, 2],
#                 'symmetry': None,
#                 'padding': 'same'
#                 }

# decoder_args = {'n_filters': [256, 128, 128, 64, 64, 3],
#                 'filter_sizes': [3, 3, 5, 5, 5, 6],
#                 'strides': [2, 2, 4, 4, 4, 8],
#                 'b_norm': b_norm,
#                 'non_linearity': tf.nn.relu,
#                 'b_norm_finish': False,
#                 'conv_op': deconv_op
#                 }

